In [87]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

#키워드 추출할 파일선택
webtoon_df = pd.read_csv('카카오페이지(썸네일추가).csv')

In [88]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma() 
        self.okt = Okt()
        #불용어사전 추가중
        with open('C:/Users/user/CodingWorkspace/stopwords.txt', 'r') as f:
            list_file = f.readlines()
        self.stopwords = list_file[0].split(",")
        
    #text를 입력받아 Kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
    def text2sentences(self, text):  
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        #text일 때 문장별로 리스트 만듦
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                                      if noun not in self.stopwords and len(noun) >1])) # 한글자 제거
        return nouns

In [89]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []

    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [90]:
class Rank(object):
    
    def get_ranks(self, graph, d=0.85): 
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 
            link_sum = np.sum(A[:,id]) 
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) 
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [91]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        
        # 키워드 수 조정
    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]

        ######################
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords

In [92]:
#크롤링한 csv파일에서 추출
title_list=[]
keyword_list=[]
writer_list=[]
genre_list=[]
desc_list=[]
url_list=[]
img_list=[]
platform_list=[]
#장르+스토리에서 키워드뽑기
for i in webtoon_df.index:
    text =webtoon_df.loc[i, '장르']+webtoon_df.loc[i, '줄거리']
    
    #print(text)
    textrank = TextRank(text)
    title_list.append(webtoon_df.loc[i, '제목'])
    writer_list.append(webtoon_df.loc[i, '작가'])
    genre_list.append(webtoon_df.loc[i, '장르'])
    desc_list.append(webtoon_df.loc[i, '줄거리'])
    url_list.append(webtoon_df.loc[i, 'url'])
    img_list.append(webtoon_df.loc[i, '썸네일'])
    platform_list.append(webtoon_df.loc[i, '플랫폼'])
    keyword_list.append(textrank.keywords())

In [93]:
web_data = pd.DataFrame()
web_data['제목'] = title_list
web_data['작가']=writer_list
web_data['장르']=genre_list
web_data['줄거리']=desc_list

web_data['url']=url_list
web_data['썸네일']=img_list
web_data['플랫폼']=platform_list

web_data['키워드'] = keyword_list
web_data.to_csv('카카오페이지_keyword.csv', encoding='utf-8-sig')

In [95]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


data = pd.read_csv('전체플랫폼_keyword.csv', low_memory=False)
data.head(10)

,Unnamed: 0,제목,작가,장르,줄거리,url,썸네일,플랫폼,키워드
0,0,참교육,채용택 / 한가람,"스토리, 액션",무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['부활', '신석기', '채용', '한가람', '교육']"
1,1,신의 탑,SIU,"스토리, 판타지",자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑,https://comic.naver.com/webtoon/list?titleId=1...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['소녀', '소년', '시험', '판타지']"
2,2,뷰티풀 군바리,설이 / 윤성원,"스토리, 드라마",'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['군대', '드라마', '만화', '여자도']"
3,3,윈드브레이커,조용석,"스토리, 스포츠",혼자서 자전거를 즐겨타던 모범생 조자현.\n원치 않게 자전거 크루의 일에 자꾸 휘말...,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['스포츠', '드라마', '라이', '말리', '모범생']"
4,4,퀘스트지상주의,박태준 만화회사,"스토리, 드라마","[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\n공부, 싸...",https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['퀘스트', '작품', '공부', '싸움', '외모']"
5,5,쇼미더럭키짱!,박태준 / 김성모,"에피소드, 액션",고작 18살 나이로 부산을 꿇린 남자 강건마\n메마른 그의 가슴을 송두리째 불태울 ...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['액션', '통합', '남자', '래퍼', '사나이']"
6,6,장씨세가 호위무사,김인호 / 조형근,"스토리, 액션",‘당신이 부른 것이오. 나란 사람을... ’\n은둔고수 광휘. 호위무사 되다.\n웹...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['고수', '광휘', '무사', '무협', '시대극']"
7,7,소녀의 세계,모랑지,"스토리, 드라마",완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나\n여러 갈등을 함께 ...,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['드라마', '맘씨', '오리가', '갈등', '백조']"
8,8,백수세끼,치즈,"스토리, 드라마",백수 시절 내 곁을 지켜줬던 그녀... 돌아와 주면 안 되겠니?\n음식 메뉴마다 담...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['드라마', '백수', '시절', '메뉴', '연애']"
9,9,유미의 작가 수칙,이동건,"스토리, 일상",본 컨텐츠는 이동건 작가가 전하는 네이버의 브랜드 웹툰입니다.\n프로 작가의 삶을 ...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['브랜드', '일상', '전하', '컨텐츠', '툰입니']"


In [96]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['키워드'])
print('TF-IDF 행렬 :',tfidf_matrix.shape)

TF-IDF 행렬 : (8554, 8555)


In [99]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['키워드'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 행렬:',cosine_sim.shape)
# print(cosine_sim)

코사인 유사도 행렬: (8554, 8554)


In [103]:
title_to_index = dict(zip(data['제목'], data.index))

index = title_to_index['참교육']


In [104]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

def get_recommendations(text, cosine_sim=cosine_sim):
    index = title_to_index[text]
    
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)

    web_indices = [index[0] for index in sim[1:6]]
    
    print(data['제목'].iloc[web_indices])
    
    print(str(cosine_sim[index][web_indices]))
    
    #코사인 유사도

In [105]:
get_recommendations('참교육')

1213           부활남
7302    찬란한 액션 유치원
1379       오늘부터 주군
4039          살신성인
1194      용비불패 완전판
Name: 제목, dtype: object
[0.32875493 0.21325185 0.20747017 0.19337654 0.19293086]
